# 分析 TVM 自动量化结果

In [ ]:
import numpy as np
from tvm import relay
import tvm
from tvm_book.tvm_utils.llvm_utils import run_llvm_graph
from tqdm import tqdm
from tvm_book.tvm_utils.relay_pattern import *

def load_model(input_shape=[1, 3, 224, 224]):
    """加载前端模型"""
    import torch
    from torchvision.models import mobilenet_v2 as model
    from torchvision.models.mobilenet import MobileNet_V2_Weights
    model = model(weights=MobileNet_V2_Weights.IMAGENET1K_V1)
    data = torch.randn(*input_shape)
    return torch.jit.trace(model.eval(), data)

size = 224, 224
input_shape = (1, 3, *size)
input_name = "data"
traced_model = load_model(input_shape).eval()
# 将前端模型翻译为 relay 模型
origin_mod, origin_params = relay.frontend.from_pytorch(traced_model, [(input_name, input_shape)])

先以 `mod` 子图为例研究定义量化过程：

In [ ]:
from tvm_book.tvm_utils.split_graph import graph_split

split_conf = [{"op_name": "clip", "op_index": 1}]
mod = graph_split(origin_mod["main"], split_conf)[0]
print(mod["main"])

记录量化过程：

In [ ]:
# 定义校准数据集
def data_iter(input_name, input_shape, num=1):
    for _ in range(num):
        yield {input_name: np.random.normal(size=input_shape)}

dataset = data_iter(input_name, input_shape)

calibrate_pass = tvm.transform.module_pass(
    relay.quantize.calibrate(dataset), opt_level=1, name="QuantizeCalibrate"
)

with tvm.transform.PassContext(opt_level=3):
    with relay.quantize.qconfig(
        calibrate_mode="kl_divergence",
        weight_scale="max",
        skip_conv_layers=[],
        skip_dense_layer=False
    ):
        # 量化前准备
        opt_mod = relay.quantize.prerequisite_optimize(mod, origin_params)
        # 划分计算图
        partition_mod = relay.quantize.partition()(opt_mod)
        # 注解计算图
        annotate_mod = relay.quantize.annotate()(partition_mod)
        # 校准量化
        calibrate_mod = calibrate_pass(annotate_mod)
        # 量化实现
        qmod = relay.quantize.realize()(calibrate_mod)

定义融合规则：

In [ ]:

# 配置融合规则
compiler_name = "ccompiler"
pattern_table = [
    (f"{compiler_name}.conv_add_relu_max_pool2d", make_conv_add_relu_max_pool2d_pattern()),
    (f"{compiler_name}.conv2d_transpose_add_activate", make_conv2d_transpose_add_activate_pattern()),
    (f"{compiler_name}.conv_add_activate", make_conv_add_activate_pattern()),
    (f"{compiler_name}.max_pool2d", make_max_pool2d_pattern()),
    (f"{compiler_name}.dense_add", make_dense_add_pattern()),
    (f"{compiler_name}.adaptive_avg_pool2d", make_adaptive_avg_pool2d_pattern()),
    (f"{compiler_name}.avg_pool2dd", make_avg_pool2d_pattern()),
    (f"{compiler_name}.add_multiply_add", make_add_multiply_add_pattern()),
    (f"{compiler_name}.add", make_add_pattern()),
    (f"{compiler_name}.multiply", make_multiply_pattern()),
    # (f"{compiler_name}.strided_slice", make_strided_slice_pattern()),
]
merge_passes = tvm.transform.Sequential([
    relay.transform.InferType(),
    relay.transform.MergeComposite(pattern_table),
    # relay.transform.ToANormalForm(),
    # relay.transform.ToGraphNormalForm(),
])
with tvm.transform.PassContext(opt_level=3):
    partition_mod_f = merge_passes(partition_mod)
    calibrate_mod_f = merge_passes(calibrate_mod)

In [ ]:
print(calibrate_mod_f)

In [ ]:
print(partition_mod_f["main"])

## 部分量化

In [ ]:
from tvm.relay.testing import run_opt_pass
from tvm.relay.expr import Constant, Call, Let
from tvm.relay.function import Function, FunctionWithFields
from tvm.relay import GlobalVar
from tvm.relay.op.annotation import compiler_begin, compiler_end

class DefuseMutator(tvm.relay.ExprMutator):
    def __init__(self, mod):
        super().__init__()
        self.mod = mod
        self.bind_params = {}
    
    # def visit_let(self, let):
    #     new_var = self.visit(let.var)
    #     new_val = self.visit(let.value)
    #     new_body = self.visit(let.body)
    #     if new_var == let.var and new_val == let.value and new_body == let.body:
    #         new_let = let
    #     else:
    #         new_let = Let(new_var, new_val, new_body)
    #     return new_let

    def visit_call(self, call):
        new_fn = self.visit(call.op)
        new_args = [self.visit(arg) for arg in call.args]
        
        if isinstance(new_fn, GlobalVar):
            self.bind_params[new_fn.name_hint] = new_args
            new_fn = self.mod[new_fn.name_hint].body
            new_fn = run_opt_pass(new_fn, relay.transform.DefuseOps())
            # _bind_dict = dict(zip(new_fn.params, new_args))
            # new_fn = relay.bind(new_fn, _bind_dict)
        new_call = Call(new_fn, new_args, call.attrs, call.type_args, call.span)
        return new_call

@tvm.relay.transform.function_pass(opt_level=1)
class CalibrateGraphTransform:
    def __init__(self, compiler):
        self.reset()
        self.compiler = compiler
        
    def reset(self):
        self._func_index = 0
        self.bind_dict = {}
        self.nodes = []

    def transform_function(self, func, mod, ctx):
        obj = self
        class Replace(tvm.relay.ExprMutator):
            def visit_call(self, call):
                new_fn = self.visit(call.op)
                new_args = [self.visit(arg) for arg in call.args]
                
                if isinstance(new_fn, Function):
                    compiler = obj.compiler
                    func_name = f"f_{obj._func_index:04d}"
                    # obj.nodes.append(call)
                    new_args = [compiler_begin(arg, obj.compiler) for arg in new_args]
                    # obj.bind_dict[func_name] = new_args
                    # _bind_dict = dict(zip(new_fn.params, new_args))
                    # new_fn = relay.bind(new_fn, _bind_dict)
                    
                    # if new_fn.attrs:
                    #     name = new_fn.attrs["PartitionedFromPattern"].replace("relay.op.annotation.simulated_quantize_", "").replace("annotation.cast_hint_", "")
                    #     if "conv" in name:
                    #         data = new_args[0]
                    #         old_data = new_fn.params[0]
                    #         body = new_fn.body
                    #         if data.op.name == "relay.op.annotation.simulated_quantize":
                    #             ...
                    
                    # mod[func_name] = new_fn
                    # # # new_args = new_fn.params
                    # new_fn = mod.get_global_var(func_name)
                    # obj._func_index += 1
                else:
                    compiler = "default"
                call = Call(new_fn, new_args, call.attrs, call.type_args, call.span)
                return compiler_end(call, compiler)
        return Replace().visit(func)

@tvm.relay.transform.function_pass(opt_level=1)
class PartitionGraphTransform:
    def __init__(self):
        self.reset()
        
    def reset(self):
        self._func_index = 0
        self.bind_dict = {}

    def transform_function(self, func, mod, ctx):
        obj = self
        class Replace(tvm.relay.ExprMutator):
            def visit_call(self, call):
                new_fn = self.visit(call.op)
                new_args = [self.visit(arg) for arg in call.args]
                
                if isinstance(call.op, Function):
                    func_name = f"f_{obj._func_index:04d}"
                    _bind_dict = dict(zip(new_fn.params, new_args))
                    _new_fn = relay.bind(new_fn, _bind_dict)
                    _new_fn = run_opt_pass(_new_fn, relay.transform.DefuseOps())
                    obj.bind_dict[func_name] = _new_fn
                    obj._func_index += 1
                call = Call(new_fn, new_args, call.attrs, call.type_args, call.span)
                return call
        return Replace().visit(func)


In [ ]:
calibrate_transform = CalibrateGraphTransform("test")
# partition_transform = PartitionGraphTransform()
with tvm.transform.PassContext(opt_level=3):
    calibrate_mod_t = calibrate_transform(calibrate_mod_f)
    # partition_mod_t = partition_transform(partition_mod_f)

In [ ]:
# call = relay.Tuple(calibrate_transform.nodes)
# call = run_opt_pass(call, relay.transform.DefuseOps())

In [ ]:
run_mod = relay.transform.ToANormalForm()(calibrate_mod_f)

In [101]:
from tvm.relay.expr import Var

@tvm.relay.transform.function_pass(opt_level=1)
class CalibrateGraphTransform:
    def __init__(self, compiler):
        self.reset()
        self.compiler = compiler
        
    def reset(self):
        self._func_index = 0
        self.bind_dict = {}
        self.nodes = []

    def transform_function(self, func, mod, ctx):
        obj = self
        class Replace(tvm.relay.ExprMutator):
            def visit_call(self, call):
                new_fn = self.visit(call.op)
                new_args = [self.visit(arg) for arg in call.args]
                # print(type(new_fn))
                
                if isinstance(new_fn, Var):
                    compiler = obj.compiler
                    func_name = f"f_{obj._func_index:04d}"
                    obj.nodes.append(call)
                    new_args = [compiler_begin(arg, compiler) for arg in new_args]
                    # new_fn = run_opt_pass(new_fn, relay.transform.DefuseOps())
                    # obj.bind_dict[func_name] = new_args
                    # _bind_dict = dict(zip(new_fn.params, new_args))
                    # new_fn = relay.bind(new_fn, _bind_dict)
                    
                    # if new_fn.attrs:
                    #     name = new_fn.attrs["PartitionedFromPattern"].replace("relay.op.annotation.simulated_quantize_", "").replace("annotation.cast_hint_", "")
                    #     if "conv" in name:
                    #         data = new_args[0]
                    #         old_data = new_fn.params[0]
                    #         body = new_fn.body
                    #         if data.op.name == "relay.op.annotation.simulated_quantize":
                    #             ...
                    new_fn = compiler_begin(new_fn, compiler)
                    # mod[func_name] = new_fn
                    # # # new_args = new_fn.params
                    # new_fn = mod.get_global_var(func_name)
                    # obj._func_index += 1
                else:
                    compiler = "default"
                call = Call(new_fn, new_args, call.attrs, call.type_args, call.span)
                return compiler_end(call, compiler)
        return Replace().visit(func)
            # def visit_let(self, let):
            #     new_var = self.visit(let.var)
            #     new_val = self.visit(let.value)
            #     new_body = self.visit(let.body)
            #     if new_var == let.var and new_val == let.value and new_body == let.body:
            #         new_let = let
            #     else:
            #         new_let = Let(new_var, new_val, new_body)
            #     if isinstance(let.value, Function):
            #         func_name = f"f_{obj._func_index:04d}"
            #         obj.nodes.append(let)
            #         # mod[func_name] = tvm.IRModule.from_expr(let.body)["main"]
            #         obj._func_index += 1
            #         # new_val, new_body = new_body, new_val
            #     return new_let
        return Replace().visit(func)

In [102]:
calibrate_transform = CalibrateGraphTransform("test")
with tvm.transform.PassContext(opt_level=3):
    calibrate_mod_t = calibrate_transform(run_mod)

In [103]:
nodes = calibrate_transform.nodes
node = nodes[0]

In [104]:
print(relay.transform.PartitionGraph()(calibrate_mod_t))

InternalError: Traceback (most recent call last):
  104: tvm::runtime::PackedFuncObj::Extractor<tvm::runtime::PackedFuncSubObj<tvm::runtime::TypedPackedFunc<tvm::IRModule (tvm::transform::Pass, tvm::IRModule)>::AssignTypedLambda<tvm::transform::{lambda(tvm::transform::Pass, tvm::IRModule)#7}>(tvm::transform::{lambda(tvm::transform::Pass, tvm::IRModule)#7}, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}> >::Call(tvm::runtime::PackedFuncObj const*, tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)
  103: tvm::transform::Pass::operator()(tvm::IRModule) const
  102: tvm::transform::Pass::operator()(tvm::IRModule, tvm::transform::PassContext const&) const
  101: tvm::transform::SequentialNode::operator()(tvm::IRModule, tvm::transform::PassContext const&) const
  100: tvm::transform::Pass::operator()(tvm::IRModule, tvm::transform::PassContext const&) const
  99: tvm::transform::ModulePass

In [105]:
print(calibrate_mod_t)

def @main(%data: Tensor[(1, 3, 224, 224), float32] /* ty=Tensor[(1, 3, 224, 224), float32] span=aten::_convolution_0.data:0:0 */) -> Tensor[(1, 32, 112, 112), float32] {
  let %x_417: float32 /* ty=float32 */ = 0.0340509f /* ty=float32 */;
  let %x_418: float32 /* ty=float32 */ = -127f /* ty=float32 */;
  let %x_419: float32 /* ty=float32 */ = 127f /* ty=float32 */;
  %0 = relay.op.annotation.simulated_quantize(%data, %x_417, %x_418, %x_419, kind=1) /* ty=Tensor[(1, 3, 224, 224), float32] */;
  let %x_420: Tensor[(1, 3, 224, 224), float32] /* ty=Tensor[(1, 3, 224, 224), float32] */ = annotation.compiler_end(%0, compiler="default") /* ty=Tensor[(1, 3, 224, 224), float32] */;
  let %x_421: Tensor[(32, 3, 3, 3), float32] /* ty=Tensor[(32, 3, 3, 3), float32] */ = meta[relay.Constant][0] /* ty=Tensor[(32, 3, 3, 3), float32] */;
  let %x_422: float32 /* ty=float32 */ = 0.00283333f /* ty=float32 */;
  let %x_423: float32 /* ty=float32 */ = -127f /* ty=float32 */;
  let %x_424: float32 /* ty=f

In [ ]:
node.op

In [ ]:
func_id = 0
call = calibrate_transform.nodes[func_id]
new_fn = call.op
new_args = new_fn.params
call = run_opt_pass(call, relay.transform.DefuseOps())
print(call)
new_fn = tvm.IRModule.from_expr(call)["main"]
call = Call(new_fn.body, new_fn.params, attrs=new_fn.attrs, type_args=new_fn.type_params, span=new_fn.span)
# call = run_opt_pass(call, relay.transform.DefuseOps())
# new_args = [relay.var(f"x_{func_id:03}_{ind:03d}") for ind, arg in enumerate(new_fn.params)]
# _bind_dict = dict(zip(new_fn.params, new_args))
# new_fn = relay.bind(new_fn, _bind_dict)

In [ ]:
print(call)

In [ ]:
g_var_names = []
for arg in calibrate_mod_t.get_global_vars():
    if arg.name_hint == "main":
        continue
    new_fn = calibrate_mod_t[arg]
    # 过滤不需要的函数
    name = new_fn.attrs["PartitionedFromPattern"].replace("relay.op.annotation.simulated_quantize_", "").replace("annotation.cast_hint_", "")
    if "squeeze" in name:
        continue
    g_var_names.append(arg.name_hint)
g_var_names.sort()


In [ ]:
float_mod = tvm.IRModule()
for func_name in g_var_names:
    new_fn = partition_transform.bind_dict[func_name]
    float_mod[func_name] = new_fn

q_mod = tvm.IRModule()
for func_name in g_var_names:
    new_fn = calibrate_mod_t[func_name]
    new_fn = DefuseMutator(float_mod).visit(new_fn)
    q_mod[func_name] = new_fn

In [ ]:
# partial_quant_outputs = []
# for func_name in tqdm(g_var_names):
#     new_fn = calibrate_mod_t[func_name]
#     new_fn = DefuseMutator(float_mod).visit(new_fn)
#     run_mod = tvm.IRModule.from_expr(new_fn)
#     # partial_quant_output = run_llvm_graph(
#     #     run_mod,
#     #     origin_params,
#     #     {"data": np.random.normal(size=input_shape)}
#     # )
#     # partial_quant_outputs.append(partial_quant_output)
#     break

In [ ]:
float_mod_t = tvm.IRModule()
main_args = mod["main"].params
main_fns = []
for k, func_name in enumerate(g_var_names):
    new_fn = float_mod[func_name]
    new_args = [relay.var(f"x_{k:03}_{ind:03d}") for ind, arg in enumerate(new_fn.params)]
    _bind_dict = dict(zip(new_fn.params, new_args))
    new_fn = relay.bind(new_fn, _bind_dict)
    float_mod_t[func_name] = new_fn
    call = Call(new_fn, main_args, attrs=new_fn.attrs, type_args=new_fn.type_params, span=new_fn.span)
    main_fns.append(call)
new_fn = run_opt_pass(relay.Tuple(main_fns), relay.transform.DefuseOps())
float_mod_t = tvm.IRModule.from_expr(new_fn)

In [ ]:
float_outputs = run_llvm_graph(
    float_mod_t,
    origin_params,
    {"data": np.random.normal(size=input_shape)}
)

In [ ]:
partial_quant_outputs = run_llvm_graph(
    float_mod_t,
    origin_params,
    {"data": np.random.normal(size=input_shape)}
)

In [ ]:
partial_quant_outputs = []

for func_name in tqdm(g_var_names):
    new_fn = calibrate_mod_t[func_name]
    new_params = transform.bind_params[func_name]
    _bind_dict = dict(zip(new_fn.params, new_params))
    new_fn = relay.bind(new_fn, _bind_dict)
    new_fn = DefuseTransform(partition_mod_t).visit(new_fn)
    new_fn = run_opt_pass(new_fn, relay.transform.DefuseOps())
    print(f"部分量化 {func_name}:\n{new_fn}")
    run_mod = tvm.IRModule.from_expr(new_fn)
    # 部分量化模型输出
    partial_quant_output = run_llvm_graph(
        run_mod,
        origin_params,
        {"data": np.random.normal(size=input_shape)}
    )
    partial_quant_outputs.append(partial_quant_output[0])
    # if func_name == "f_0002":
    #     break


In [ ]:
partial_quant_outputs[0].shape

fn (%data: Tensor[(1, 3, 16, 16), float32] /* ty=Tensor[(1, 3, 16, 16), float32] span=aten::_convolution_0.data:0:0 */) -> Tensor[(1, 16, 17, 17), float32] {
  %1 = fn (%FunctionVar_0_0: Tensor[(1, 3, 16, 16), float32] /* ty=Tensor[(1, 3, 16, 16), float32] */, %FunctionVar_0_1: Tensor[(16, 3, 1, 1), float32] /* ty=Tensor[(16, 3, 1, 1), float32] */, PartitionedFromPattern="nn.conv2d_nn.max_pool2d_", Composite="ccompiler.conv_add_relu_max_pool2d") -> Tensor[(1, 16, 17, 17), float32] {
    %0 = nn.conv2d(%FunctionVar_0_0, %FunctionVar_0_1, padding=[1, 1, 1, 1], channels=16, kernel_size=[1, 1]) /* ty=Tensor[(1, 16, 18, 18), float32] span=aten::_convolution_0:0:0 */;
    nn.max_pool2d(%0, pool_size=[2, 2], padding=[0, 0, 0, 0], ceil_mode=True) /* ty=Tensor[(1, 16, 17, 17), float32] span=aten::max_pool2d_0:0:0 */
  } /* ty=fn (Tensor[(1, 3, 16, 16), float32], Tensor[(16, 3, 1, 1), float32]) -> Tensor[(1, 16, 17, 17), float32] */;
  %1(%data, meta[relay.Constant][0] /* ty=Tensor[(16, 3, 1, 1)